# Failure-Inducing Changes

_Brief abstract/introduction/motivation.  State what the chapter is about in 1-2 paragraphs._
_Then, have an introduction video:_

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Fuzzer.ipynb).

In [ ]:
import bookutils

In [ ]:
from bookutils import quiz, print_file, print_content

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.ChangeDebugger import <identifier>
```

and then make use of the following features.


_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
>>> print(int_fuzzer())
76.5
```


## A Version History

Using the `remove_html_markup()` versions from [the introduction to debugging](Intro_Debugging.ipynb) and [the chapter on assertions](Assertions.ipynb), we create a little version history.

### Create a Working Directory

In [ ]:
PROJECT = 'my_project'

In [ ]:
import os
import shutil

In [ ]:
try:
    shutil.rmtree(PROJECT)
except FileNotFoundError:
    pass
os.mkdir(PROJECT)

In [ ]:
import sys

In [ ]:
sys.path.append(os.getcwd())
os.chdir(PROJECT)

### Initialize git

In [ ]:
!git init

In [ ]:
!git config advice.detachedHead False

In [ ]:
def remove_html_markup(s):
    tag = False
    out = ""

    for c in s:
        if c == '<':    # start of markup
            tag = True
        elif c == '>':  # end of markup
            tag = False
        elif not tag:
            out = out + c

    return out

In [ ]:
import inspect

In [ ]:
def write_source(fun, filename=None):
    if filename is None:
        filename = fun.__name__ + '.py'
    with open(filename, 'w') as fh:
        fh.write(inspect.getsource(fun))

In [ ]:
write_source(remove_html_markup)

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
!git add remove_html_markup.py

In [ ]:
!git commit -m "First version"

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git diff remove_html_markup.py

In [ ]:
!git commit -m "Second version" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        print("c =", repr(c), "tag =", tag, "quote =", quote)

        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Third version (with debugging output)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<':  # and not quote:
            tag = True
        elif c == '>':  # and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Fourth version (clueless)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        assert not tag  # <=== Just added

        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Fifth version (with assert)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            assert False  # <=== Just added
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Sixth version (with another assert)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif (c == '"' or c == "'") and tag:  # <-- FIX
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Seventh version (fixed)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    # postcondition
    assert '<' not in out and '>' not in out

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Eighth version (with proper assertion)" remove_html_markup.py

We find that the latest version has an error.

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    assert remove_html_markup('"foo"') == '"foo"'

When did the error occur?

## Accessing Versions

In [ ]:
!git log --pretty=oneline

In [ ]:
import subprocess

In [ ]:
def get_output(command):
    result = subprocess.run(command, 
                            stdout=subprocess.PIPE,
                            universal_newlines=True)
    return result.stdout

In [ ]:
log = get_output(['git', 'log', '--pretty=oneline'])
print(log)

In [ ]:
versions = [line.split()[0] for line in log.split('\n') if line]
versions.reverse()

In [ ]:
!git checkout {versions[0]}

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

In [ ]:
!git checkout {versions[7]}

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

## Manual Bisecting

In [ ]:
!git bisect start

In [ ]:
!git bisect good {versions[0]}

In [ ]:
!git bisect bad {versions[7]}

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

In [ ]:
!git bisect bad

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

In [ ]:
!git bisect bad

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

In [ ]:
!git bisect bad

In [ ]:
!git diff HEAD^

In [ ]:
!git bisect view

In [ ]:
!git bisect reset

## Automatic Bisecting

In [ ]:
open('test.py', 'w').write('''
#!/usr/bin/env python

from remove_html_markup import remove_html_markup
import sys

result = remove_html_markup('"foo"')
if result == '"foo"':
    sys.exit(0)  # good/pass
elif result == 'foo':
    sys.exit(1)  # bad/fail
else:
    sys.exit(125)  # unresolved
''')
print_file('test.py')

In [ ]:
!python ./test.py; echo $?

In [ ]:
!git bisect start

In [ ]:
!git bisect good {versions[0]}

In [ ]:
!git bisect bad {versions[7]}

In [ ]:
!git bisect run python test.py

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
!git diff HEAD^

In [ ]:
!git bisect reset

## Delta Debugging on Changes

In [ ]:
from DeltaDebugger import DeltaDebugger

In [ ]:
version_1 = get_output(['git', 'show', 
                            f'{versions[0]}:remove_html_markup.py'])

In [ ]:
print_content(version_1, '.py')

In [ ]:
version_2 = get_output(['git', 'show', 
                            f'{versions[1]}:remove_html_markup.py'])

In [ ]:
print_content(version_2, '.py')

In [ ]:
!git diff {versions[0]} {versions[1]}

We use Google's [diff-match-patch library](https://github.com/google/diff-match-patch) as a [Python module](https://github.com/JoshData/diff_match_patch-python).

In [ ]:
from diff_match_patch import diff_match_patch

In [ ]:
def diff(s1, s2, mode='lines'):
    dmp = diff_match_patch()
    if mode == 'lines':
        (text1, text2, linearray) = dmp.diff_linesToChars(s1, s2)
        diffs = dmp.diff_main(text1, text2)
        dmp.diff_charsToLines(diffs, linearray)
        return dmp.patch_make(diffs)

    if mode == 'chars':
        diffs = dmp.diff_main(s1, s2)
        return dmp.patch_make(s1, diffs)
        
    raise ValueError("mode must be 'lines' or 'chars'")

In [ ]:
patches = diff(version_1, version_2)
patches

In [ ]:
import urllib

In [ ]:
def patch_string(p):
    return urllib.parse.unquote(str(p).strip())

In [ ]:
for p in patches:
    print(patch_string(p))

In [ ]:
def patch(s, patches):
    dmp = diff_match_patch()
    text, success = dmp.patch_apply(patches, s)
    assert all(success)
    return text

In [ ]:
print_content(patch(version_1, patches), '.py')

In [ ]:
assert patch(version_1, patches) == version_2

In [ ]:
assert patch(version_1, []) == version_1

In [ ]:
print(patch_string(patches[0]))

In [ ]:
print_content(patch(version_1, [patches[0]]))

In [ ]:
print_content(patch(version_1, [patches[1]]))

In [ ]:
def test_remove_html_markup(patches):
    new_version = patch(version_1, patches)
    exec(new_version, globals())
    assert remove_html_markup('"foo"') == '"foo"'

In [ ]:
test_remove_html_markup([])

In [ ]:
with ExpectError():
    test_remove_html_markup(patches)

In [ ]:
with DeltaDebugger() as dd:
    test_remove_html_markup(patches)

In [ ]:
reduced_patches = dd.reduced_args()['patches']

In [ ]:
for p in reduced_patches:
    print(urllib.parse.unquote(str(p)))

In [ ]:
print_content(patch(version_1, reduced_patches), '.py')

## A ChangeDebugger class

```python
with ChangeDebugger(source, patches):
    foo()
```

```python
exec(source); foo() -> PASS
bad_source = patch(source); exec(bad_source); foo() -> FAIL
```

or better:

```python
with ChangeDebugger(pass_source, fail_source):
    foo()
```

Implement general dd() here. Consider extending DeltaDebugger such that it not only has `reduced_args()`, but also `maximized_args()`, and `minimal_arg_diff()`.

### Excursion: All the Details

This text will only show up on demand (HTML) or not at all (PDF). This is useful for longer implementations, or repetitive, or specialized parts.

### End of Excursion

## _Section 3_

\todo{Add}

_If you want to introduce code, it is helpful to state the most important functions, as in:_

* `random.randrange(start, end)` - return a random number [`start`, `end`]
* `range(start, end)` - create a list with integers from `start` to `end`.  Typically used in iterations.
* `for elem in list: body` executes `body` in a loop with `elem` taking each value from `list`.
* `for i in range(start, end): body` executes `body` in a loop with `i` from `start` to `end` - 1.
* `chr(n)` - return a character with ASCII code `n`

In [ ]:
import random

In [ ]:
def int_fuzzer():
    """A simple function that returns a random integer"""
    return random.randrange(1, 100) + 0.5

In [ ]:
# More code
pass

## _Section 4_

\todo{Add}

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

In [ ]:
print(int_fuzzer())

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_